In [1]:
import pyspark.sql.functions as F
from pyspark.sql import SparkSession

In [2]:
spark = SparkSession \
        .builder \
        .master('local') \
        .appName('restaurants') \
        .config('spark.jars.packages', 'org.mongodb.spark:mongo-spark-connector_2.11:2.4.2') \
        .config('spark.mongodb.input.uri', 'mongodb://mongo/test.rests') \
        .config('spark.mongodb.output.uri', 'mongodb://mongo/test.rests') \
        .getOrCreate()

In [4]:
DF = spark.read.format('mongo').load()

Py4JJavaError: An error occurred while calling o95.load.
: com.mongodb.MongoTimeoutException: Timed out after 30000 ms while waiting to connect. Client view of cluster state is {type=UNKNOWN, servers=[{address=mongo:27017, type=UNKNOWN, state=CONNECTING, exception={com.mongodb.MongoSocketException: mongo}, caused by {java.net.UnknownHostException: mongo}}]
	at com.mongodb.internal.connection.BaseCluster.getDescription(BaseCluster.java:182)
	at com.mongodb.internal.connection.SingleServerCluster.getDescription(SingleServerCluster.java:41)
	at com.mongodb.client.internal.MongoClientDelegate.getConnectedClusterDescription(MongoClientDelegate.java:152)
	at com.mongodb.client.internal.MongoClientDelegate.createClientSession(MongoClientDelegate.java:103)
	at com.mongodb.client.internal.MongoClientDelegate$DelegateOperationExecutor.getClientSession(MongoClientDelegate.java:284)
	at com.mongodb.client.internal.MongoClientDelegate$DelegateOperationExecutor.execute(MongoClientDelegate.java:188)


In [ ]:
DF.printSchema()

In [5]:
df = DF.withColumn('rating__mean', DF['rating.rating__mean']) \
        .withColumn('rating__mean', DF['rating.rating__mean']) \
        .withColumn('rating__service', DF['rating.rating__service']) \
        .withColumn('rating__price_quality', DF['rating.rating__price_quality']) \
        .withColumn('rating__atmosphere', DF['rating.rating__atmosphere']).drop('rating')

In [10]:
set(df.select('district').collect())

{Row(district='адмиралтейский'),
 Row(district='василеостровский'),
 Row(district='всеволожский'),
 Row(district='выборгский'),
 Row(district='калининский'),
 Row(district='киришский'),
 Row(district='кировский'),
 Row(district='колпинский'),
 Row(district='красногвардейский'),
 Row(district='красносельский'),
 Row(district='кронштадтский'),
 Row(district='курортный'),
 Row(district='ленинского'),
 Row(district='ломоносовский'),
 Row(district='металургійний'),
 Row(district='московский'),
 Row(district='невский'),
 Row(district='петроградский'),
 Row(district='петродворцовый'),
 Row(district='приморский'),
 Row(district='пушкинский'),
 Row(district='тосненский'),
 Row(district='фрунзенский'),
 Row(district='центральный'),
 Row(district=None)}

In [11]:
df[df['district'].isNull()].select('address').collect()

[Row(address='ул. Гражданская, 27 Вознесенский пр-т, 30, Санкт-Петербург 190031 Россия'),
 Row(address='Литейный проспект 10 Рядом с Radisson Sonya Hotel, Санкт-Петербург 191028 Россия'),
 Row(address='ул. Почтамтская, д. 1, Санкт-Петербург 190000 Россия'),
 Row(address='ул. Садовая,  41, Санкт-Петербург 190031 Россия'),
 Row(address='ул. Сытнинская, д. 10 м. Горьковская, Санкт-Петербург 197101 Россия'),
 Row(address='ул. Оранжерейная, 21A, Пушкин, Санкт-Петербург 196601 Россия'),
 Row(address='наб. кан. Грибоедова, 56-58, Санкт-Петербург 190031 Россия'),
 Row(address='ул. Кирочная д. 3, Санкт-Петербург 191028 Россия'),
 Row(address='ул. Малая Конюшенная, 14, Санкт-Петербург 191186 Россия'),
 Row(address='ул. Большая Морская, 27, Санкт-Петербург 190000 Россия'),
 Row(address='Реки Мойки Набережная, д. 14, Санкт-Петербург 191186 Россия'),
 Row(address='просп. Невский, 61 2 этаж, Санкт-Петербург 191025 Россия'),
 Row(address='Ул. Конюшенная, д. 39/39, Пушкин, Санкт-Петербург 196601 Росси

In [92]:
df.filter(df['district'].isNull()).select('address').first()

Row(address='Кирочная ул., 5, Санкт-Петербург 191028 Россия')

In [13]:
df.filter(df['address'].isNotNull()).count()

9860

In [16]:
df.filter(df['check_group'].isNull()).count()

4730

In [17]:
df.filter(df['check_group'].isNull()).show()

+--------------------+--------------------+-----------+--------------------+--------------------+------------------+------------+---------------+---------------------+------------------+
|                 _id|             address|check_group|            district|       kitchen_types|              name|rating__mean|rating__service|rating__price_quality|rating__atmosphere|
+--------------------+--------------------+-----------+--------------------+--------------------+------------------+------------+---------------+---------------------+------------------+
|[5f010a5c96765359...|Александровский П...|       null| Петроградский район|[Итальянская, Аме...|         Telepizza|         5.0|           null|                 null|              null|
|[5f010a5c96765359...|                null|       null|                null|                null|              null|        null|           null|                 null|              null|
|[5f010a5c96765359...|                null|       null|          